In [4]:
import pandas as pd
import urllib,json
from pandas.io.json import json_normalize
import os
import inquirer
from datetime import datetime
from sklearn import preprocessing
from PySAM.PySSC import PySSC

ModuleNotFoundError: No module named 'PySAM'

In [5]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from time import strptime
import calendar

In [6]:
def API_Reader(url):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    json_load = json.loads(response.read())
    return(json_load)

In [7]:
eia_api_key = 'b0cbfe0b334c7c98b857879d54226b4a'
ei_api_key = 'o8dCYVBSkm1c2tnB5YUAYsKv4iZaC8AkyPYMYSBL'

# Rate Pull

In [8]:
Pay_Structure_DF = pd.DataFrame()
y = range(0,100)
for x in y:
    print(x),
    if x == 0:
        offset=0
    if x ==1:
        offset=501
    if x>1:
        offset=501 +((x-1)*500)
    api_type = 'csv'
    EI_url = 'https://api.openei.org/utility_rates?version=latest&format='+api_type+'&limit=500&sector=Industrial&detail=full&offset='+str(offset)+'&api_key=' + ei_api_key
    EI_API_CSV = pd.read_csv(EI_url)
    Pay_Structure_DF = pd.concat([Pay_Structure_DF,EI_API_CSV], axis=0, join='outer', ignore_index=True, keys=None,levels=None, names=None, verify_integrity=False, copy=True)
    if EI_API_CSV.shape[0] < 500:
            break
Pay_Structure_DF.to_csv('Pay_Structure.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [16]:
base_columns = ['label','uri',
    'approved',
    'is_default',
    'name',
    'startdate',
    'enddate',
    'utility',
    'eiaid',
    'country',
    'sector',
    'servicetype',
    'description',
    'source',
    'sourceparent',
    'revisions']
coincident_columns = list(Pay_Structure_DF.columns[Pay_Structure_DF.columns.str.contains('coincident')])# + ['coincidentratestructure/period2/tier0rate']
needed_columns = base_columns + coincident_columns
Coincident_DF = Pay_Structure_DF.loc[Pay_Structure_DF['coincidentrateunit'][Pay_Structure_DF['coincidentrateunit'].notna()].index,needed_columns ]
Coincident_DF.reset_index(inplace = True,drop = True)
Coincident_DF.shape

(150, 309)

In [28]:
coincidentschedulecolumns = Coincident_DF.columns[Coincident_DF.columns.str.contains('coincidentschedule')]
id_columns = [x for x in needed_columns if x not in coincidentschedulecolumns]
Coincident_DF_Schedule = pd.melt(Coincident_DF,id_vars = id_columns,value_vars = coincidentschedulecolumns,var_name = 'month_hour',value_name = 'schedule')
Coincident_DF_Schedule.shape

(43200, 23)

In [29]:
Coincident_DF_Schedule['month'] = Coincident_DF_Schedule.month_hour.str.split('_').str[1]
Coincident_DF_Schedule['hour'] = Coincident_DF_Schedule.month_hour.str.split('_').str[2]
d= {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9,
       'oct':10, 'nov':11, 'dec':12}
Coincident_DF_Schedule['month'].replace(d,inplace = True)

In [30]:
Coincident_DF_Schedule.loc[Coincident_DF_Schedule['schedule'] == 0,'price'] = Coincident_DF_Schedule['coincidentratestructure/period0/tier0rate']
Coincident_DF_Schedule.loc[Coincident_DF_Schedule['schedule'] == 1,'price'] = Coincident_DF_Schedule['coincidentratestructure/period1/tier0rate']
Coincident_DF_Schedule.loc[Coincident_DF_Schedule['schedule'] == 2,'price'] = Coincident_DF_Schedule['coincidentratestructure/period2/tier0rate']
Coincident_DF_Schedule.hour = Coincident_DF_Schedule.hour.astype(int)
Coincident_DF_Schedule.month = Coincident_DF_Schedule.month.astype(int)
Coincident_DF_Schedule.sort_values(by = ['utility','month','hour'],inplace = True)
Coincident_DF_Schedule.reset_index(inplace = True,drop = True)
CP_Utilities = Coincident_DF.eiaid.unique()

In [31]:
len(Coincident_DF_Schedule.utility.unique())
Coincident_DF_Schedule['utility'].unique()

array(['Access Energy Coop', 'Agralite Electric Coop',
       'Bangor Hydro-Electric Co', 'Bon Homme Yankton El Assn, Inc',
       'Butler County Rural Elec Coop', 'C & L Electric Coop Corp',
       'Carroll County REMC', 'Chippewa Valley Electric Coop',
       'City of Clintonville, Wisconsin (Utility Company)',
       'City of Fort Collins, Colorado (Utility Company)',
       'City of Gonzales, Texas (Utility Company)',
       'City of Lexington, North Carolina (Utility Company)',
       'City of Memphis, Tennessee (Utility Company)',
       'City of Monroe, North Carolina (Utility Company)',
       'City of Murfreesboro, Tennessee (Utility Company)',
       'City of Rocky Mount, North Carolina (Utility Company)',
       'City of Stanton, Iowa (Utility Company)',
       'City of Thief River Falls, Minnesota (Utility Company)',
       'City of Volga, South Dakota (Utility Company)',
       'City of Webster City, Iowa (Utility Company)',
       'Clay County Electric Coop Corp', 'Coast 

# Utility Rules

In [32]:
Utitility_Rules = pd.read_csv('https://raw.githubusercontent.com/awendell989/EIA_2018_Utiltiy_Policies/master/Dynamic_Pricing_2018_Avery.csv')
Utitility_Rules.loc[:,Utitility_Rules.columns[Utitility_Rules.columns.str.contains('--')]] = Utitility_Rules.loc[:,Utitility_Rules.columns[Utitility_Rules.columns.str.contains('--')]].replace({'N':False,'Y':True})
Industrial_Columns = Utitility_Rules.select_dtypes(include='bool').columns
Industrial_Columns = Industrial_Columns[Industrial_Columns.str.contains('Industrial')]
Utitility_Rules['Industrial_Total'] = Utitility_Rules[Industrial_Columns].sum(axis = 1)
CP_Options = Industrial_Columns + ['Industrial_Total']

In [33]:
Utitility_Rules.head()

,Data Year,Utility Number,Utility Name,State,BA Code,Residential -- Number of Customers Enrolled,Commercial -- Number of Customers Enrolled,Industrial -- Number of Customers Enrolled,Transportation -- Number of Customers Enrolled,Total -- Number of Customers Enrolled,...,Transportation -- Variable Peak Pricing,Residential -- Critical Peak Pricing,Commercial -- Critical Peak Pricing,Industrial -- Critical Peak Pricing,Transportation -- Critical Peak Pricing,Residential -- Critical Peak Rebate,Commercial -- Critical Peak Rebate,Industrial -- Critical Peak Rebate,Transportation -- Critical Peak Rebate,Industrial_Total
0,2018,3503,"Choptank Electric Coop, Inc",MD,PJM,0,213,9,.,222,...,False,False,True,True,False,False,False,False,False,2
1,2018,7027,City of Gastonia - (NC),NC,DUK,0,37,12,0,49,...,False,False,True,True,False,False,False,False,False,2
2,2018,108,Adams-Columbia Electric Coop,WI,MISO,835,939,9,.,"1,783",...,False,False,True,True,False,False,False,False,False,3
3,2018,7264,"Glades Electric Coop, Inc",FL,SEC,3,"3,421",21,.,"3,445",...,False,False,True,True,False,False,True,False,False,2
4,2018,7801,Gulf Power Co,FL,SOCO,"20,116","1,020",91,0,"21,227",...,False,True,True,True,False,False,False,False,False,3


In [34]:
#Coincident_DF = Coincident_DF.merge(Utitility_Rules[['Utility Number','BA Code']],left_on = 'eiaid',right_on = 'Utility Number')
Coincident_DF_Schedule_Full = Coincident_DF_Schedule.merge(Utitility_Rules[['Utility Number','BA Code']],left_on = 'eiaid',right_on = 'Utility Number')
CP_BAs = Coincident_DF_Schedule_Full['BA Code'].unique()

In [35]:
len(Coincident_DF_Schedule_Full.eiaid.unique())

20

# Demand Data

In [36]:
demand_total_url = 'http://api.eia.gov/category/?api_key=b0cbfe0b334c7c98b857879d54226b4a&category_id=2122628'
json_load = API_Reader(demand_total_url)
Categories = pd.DataFrame(json_load['category']['childcategories'])
Categories['BA_Code'] = Categories['name'].apply(lambda r: r.split('(')[1].split(')')[0])

In [37]:
Categories = Categories.loc[Categories['BA_Code'].isin(CP_BAs),:].reset_index(drop = True)
Categories

,category_id,name,BA_Code
0,3389963,Duke Energy Progress East (CPLE),CPLE
1,3389980,"Midcontinent Independent System Operator, Inc....",MISO
2,3389988,"PJM Interconnection, LLC (PJM)",PJM
3,3389994,Public Service Company of Colorado (PSCO),PSCO
4,3390005,"Southern Company Services, Inc. - Trans (SOCO)",SOCO
5,3390006,Southwest Power Pool (SWPP),SWPP
6,3390009,Tennessee Valley Authority (TVA),TVA
7,3390014,Western Area Power Administration - Rocky Moun...,WACM


In [38]:
Demand_Data_Total = pd.DataFrame(columns = ['UTC', 'MWh', 'BA', 'CID'])
for x in range(0,max(Categories.index+1)):
    name = Categories.iloc[x,1]
    category_id = Categories.iloc[x,0]
    category_url ='http://api.eia.gov/category/?api_key='+ str(eia_api_key) +'&category_id='+str(category_id)
    BA_Data = API_Reader(category_url)
    Series = pd.DataFrame(BA_Data['category']['childseries'])
    Series_id = Series.loc[Series['name'].str.contains('UTC'),'series_id'][0]
    series_url = 'https://api.eia.gov/series/?api_key='+eia_api_key+'&series_id='+str(Series_id)
    Series_Data = API_Reader(series_url)
    step1 = Series_Data['series'][0]
    step2 = step1['data']
    Columns = ['UTC','MWh']
    DF = pd.DataFrame(step2,columns = Columns)
    DF['BA'] = name
    DF['CID'] = category_id
    Demand_Data_Total = pd.concat([Demand_Data_Total,DF], axis=0, join='outer', ignore_index=True, keys=None,levels=None, names=None, verify_integrity=False, copy=True)
    print(name + ' added!')
    Demand_Data_Total['UTC'] = pd.to_datetime(Demand_Data_Total['UTC'],utc=True) #Is this True DOh?
    Demand_Data_Total['BA Code'] = Demand_Data_Total['BA'].apply(lambda r: r.split('(')[1].split(')')[0])

Duke Energy Progress East (CPLE) added!
Midcontinent Independent System Operator, Inc. (MISO) added!
PJM Interconnection, LLC (PJM) added!
Public Service Company of Colorado (PSCO) added!
Southern Company Services, Inc. - Trans (SOCO) added!
Southwest Power Pool (SWPP) added!
Tennessee Valley Authority (TVA) added!
Western Area Power Administration - Rocky Mountain Region (WACM) added!


In [39]:
Demand_Data_Total['hour'] = Demand_Data_Total['UTC'].dt.hour
Demand_Data_Total['hour_p'] = Demand_Data_Total['UTC'].dt.hour/24
Demand_Data_Total['month'] = Demand_Data_Total['UTC'].dt.month
Demand_Data_Total['year'] = Demand_Data_Total['UTC'].dt.year
Demand_Data_Total['day'] = Demand_Data_Total['UTC'].dt.day
Demand_Data_Total['Rank'] = Demand_Data_Total.groupby(['BA','month','year'],axis = 0)['MWh'].rank(method = 'max',ascending = False)
Demand_Data_Total['Day_Hour'] = Demand_Data_Total['day'] + Demand_Data_Total['hour_p']
current_month = datetime.now().month
current_year = datetime.now().year
Demand_Data_Total = Demand_Data_Total[~((Demand_Data_Total['month'] == current_month) & (Demand_Data_Total['year'] == current_year))]
Demand_Data_Total = Demand_Data_Total.loc[Demand_Data_Total['BA Code'].isin(CP_BAs),:]

In [40]:
Month_Min_Max = Demand_Data_Total.groupby(['BA','year','month'],axis = 0).agg({'MWh' : ['min','max']}).reset_index()
Demand_Data_Total = Demand_Data_Total.merge(Month_Min_Max,on =['BA','year','month'])
Demand_Data_Total.rename(columns = {Demand_Data_Total.columns[-1] : 'm_max',Demand_Data_Total.columns[-2] : 'm_min'},inplace = True)
Demand_Data_Total['MWH_M_Scale'] = Demand_Data_Total.apply(lambda x: (x['MWh'] - x['m_min']) / (x['m_max'] - x['m_min']),axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3878: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [41]:
Demand_Data_Total = Demand_Data_Total.loc[~((Demand_Data_Total['month'] == 2) & (Demand_Data_Total['day'] == 29)) ,]
Time_DF = Demand_Data_Total.groupby(['month','day','hour']).size().reset_index().drop(0,axis =1)

In [42]:
Demand_Data_Total.to_csv('Demand_by_BA.csv')

In [43]:
#Demand_Data_Total_Agg = Demand_Data_Total.groupby(['BA', 'BA Code', 'CID','month','day','hour','Day_Hour'])['MWH_M_Scale','MWh'].sum()
Demand_Data_Total_Agg = Demand_Data_Total.groupby(['BA', 'BA Code', 'CID','month','hour'])['MWH_M_Scale','MWh'].mean()
Demand_Data_Total_Agg.reset_index(inplace = True)
Demand_Data_Total_Agg.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,BA,BA Code,CID,month,hour,MWH_M_Scale,MWh
0,Duke Energy Progress East (CPLE),CPLE,3389963,1,0,0.439112,8085.116129
1,Duke Energy Progress East (CPLE),CPLE,3389963,1,1,0.444427,8130.135484
2,Duke Energy Progress East (CPLE),CPLE,3389963,1,2,0.439408,8089.987097
3,Duke Energy Progress East (CPLE),CPLE,3389963,1,3,0.415967,7896.367742
4,Duke Energy Progress East (CPLE),CPLE,3389963,1,4,0.377323,7576.845161


# Merge to Rate Schedule

In [44]:
Demand_Rate_Schedule = Demand_Data_Total_Agg.merge(Coincident_DF_Schedule_Full, how='right', left_on=['month','hour','BA Code'], right_on=['month','hour','BA Code'])
Demand_Rate_Schedule = Demand_Rate_Schedule[['BA Code','utility','eiaid','name','month','hour','MWh','MWH_M_Scale','schedule','price']].sort_values(by =['BA Code','utility','eiaid','name','month','hour'] ).reset_index(drop = True)

In [45]:
Demand_Rate_Schedule.to_csv('Demand_Rate_Schedule.csv',index = False)

In [46]:
len(Demand_Rate_Schedule.eiaid.unique())

20

# Zipcodes

In [47]:
U_to_Z_I = pd.read_csv('iouzipcodes2011.csv',dtype = {'zip': str})
U_to_Z_NI = pd.read_csv('noniouzipcodes2011.csv',dtype = {'zip': str})

In [48]:
#Concat And Run

U_to_Z = pd.concat([U_to_Z_I,U_to_Z_NI])
U_to_Z = U_to_Z.drop_duplicates(subset=['zip','eiaid'],keep = 'last').sort_values(by = 'zip')
#U_to_Z_Zips = U_to_Z.loc[U_to_Z['eiaid'].isin(CP_Utilities),'zip']
file = os.path.join(os.getcwd(),'CP_Zips.csv')
U_to_Z.to_csv(file,index = False )

In [50]:
U_to_Z.head()

,zip,eiaid,utility_name,state,service_type,ownership,comm_rate,ind_rate,res_rate
6819,00501,11171,Long Island Power Authority,NY,Delivery,State,0.078384,0.000000,0.200000
6912,00544,11171,Long Island Power Authority,NY,Delivery,State,0.078384,0.000000,0.200000
35987,01001,20455,Western Massachusetts Elec Co,MA,Delivery,Investor Owned,0.067246,0.050643,0.076298
35985,01002,20455,Western Massachusetts Elec Co,MA,Delivery,Investor Owned,0.067246,0.050643,0.076298
35986,01003,20455,Western Massachusetts Elec Co,MA,Delivery,Investor Owned,0.067246,0.050643,0.076298


In [51]:
Demand_Rate_Zip_Schedule = Demand_Rate_Schedule.merge(U_to_Z[['zip','eiaid']],on ='eiaid',how ='left')
Demand_Rate_Zip_Schedule.sort_values(by =['BA Code','utility','eiaid','name','zip','month','hour'],inplace = True)
Demand_Rate_Zip_Schedule.dropna(inplace = True)
Demand_Rate_Zip_Schedule.reset_index(inplace = True,drop = True)
Demand_Rate_Zip_Schedule.head()

,BA Code,utility,eiaid,name,month,hour,MWh,MWH_M_Scale,schedule,price,zip
0,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,0,8085.116129,0.439112,0.0,15.3,27525
1,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,1,8130.135484,0.444427,0.0,15.3,27525
2,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,2,8089.987097,0.439408,0.0,15.3,27525
3,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,3,7896.367742,0.415967,0.0,15.3,27525
4,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,4,7576.845161,0.377323,0.0,15.3,27525


In [52]:
len(Demand_Rate_Zip_Schedule.eiaid.unique())

18

In [53]:
CP_Zips = Demand_Rate_Zip_Schedule.zip.unique()

# Solar Power

In [54]:
Weather_Files = pd.DataFrame(os.listdir(os.path.join(os.getcwd(),'Weather_Files')),columns = ['Loc'])
Weather_Files = pd.concat([Weather_Files,Weather_Files.Loc.str.split('_',expand = True).rename(columns = {0:'Zip',1:'Lat',2:'Long'})],axis = 1)
Weather_Files_Zips = Weather_Files.Zip
Zip_Daily_Hourly_Solar_Production = pd.DataFrame()

In [55]:
Need_WF_Zips = list(set(CP_Zips) - set(Weather_Files_Zips))
for x in Need_WF_Zips:
    print(x)

In [56]:
len(CP_Zips)

645

In [57]:
Weather_Files

,Loc,Zip,Lat,Long,3,4,5
0,15909_40.398047_-78.866430_psmv3_60_tmy.csv,15909,40.398047,-78.866430,psmv3,60,tmy.csv
1,16922_41.682698_-77.701532_psmv3_60_tmy.csv,16922,41.682698,-77.701532,psmv3,60,tmy.csv
2,01255_42.108657_-73.120833_psmv3_60_tmy.csv,01255,42.108657,-73.120833,psmv3,60,tmy.csv
3,19438_40.267083_-75.388577_psmv3_60_tmy.csv,19438,40.267083,-75.388577,psmv3,60,tmy.csv
4,17255_40.167554_-77.867235_psmv3_60_tmy.csv,17255,40.167554,-77.867235,psmv3,60,tmy.csv
...,...,...,...,...,...,...,...
1357,16662_40.296693_-78.313891_psmv3_60_tmy.csv,16662,40.296693,-78.313891,psmv3,60,tmy.csv
1358,19013_39.847986_-75.374864_psmv3_60_tmy.csv,19013,39.847986,-75.374864,psmv3,60,tmy.csv
1359,81416_38.736655_-108.099446_psmv3_60_tmy.csv,81416,38.736655,-108.099446,psmv3,60,tmy.csv
1360,16403_41.793755_-80.030490_psmv3_60_tmy.csv,16403,41.793755,-80.030490,psmv3,60,tmy.csv


In [58]:
for m in CP_Zips:
    print(m)
    Loc = Weather_Files.loc[Weather_Files['Zip'] == m,'Loc'].values[0]
    file = os.path.join(os.getcwd(),'Weather_Files',Loc)
    ssc = PySSC()
    print ('Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2')
    print ('SSC Version = ', ssc.version())
    ssc.module_exec_set_print(0)
    data = ssc.data_create()
    ssc.data_set_string( data, b'solar_resource_file',str.encode(file));  #'b/Users/Avery/Desktop/Coincident_Peak_Project/Weather_Files/17302_39.810052_-76.408374_psmv3_60_tmy.csv' );
    ssc.data_set_number( data, b'system_capacity', 10000 )
    ssc.data_set_number( data, b'module_type', 0 )
    ssc.data_set_number( data, b'dc_ac_ratio', 1.1499999761581421 )
    ssc.data_set_number( data, b'inv_eff', 96 )
    ssc.data_set_number( data, b'losses', 14.075660705566406 )
    ssc.data_set_number( data, b'array_type', 0 )
    ssc.data_set_number( data, b'tilt', 20 )
    ssc.data_set_number( data, b'azimuth', 180 )
    ssc.data_set_number( data, b'gcr', 0.40000000596046448 )
    ssc.data_set_number( data, b'adjust:constant', 0)
    module = ssc.module_create(b'pvwattsv5')
    ssc.module_exec_set_print( 0 );
    if ssc.module_exec(module, data) == 0:
        print ('pvwattsv5 simulation error')
        idx = 1
        msg = ssc.module_log(module, 0)
        while (msg != None):
            print ('	: ' + msg.decode("utf - 8"))
            msg = ssc.module_log(module, idx)
            idx = idx + 1
        SystemExit( "Simulation Error" );
    ssc.module_free(module)
    annual_energy = ssc.data_get_number(data, b'annual_energy');
    print ('Annual energy (year 1) = ', annual_energy)
    capacity_factor = ssc.data_get_number(data, b'capacity_factor');
    print ('Capacity factor (year 1) = ', capacity_factor)
    kwh_per_kw = ssc.data_get_number(data, b'kwh_per_kw');
    print ('Energy yield (year 1) = ', kwh_per_kw)
    hourly_data = pd.Series(ssc.data_get_array(data,b'gen'))
    hourly_data = pd.concat([Time_DF,hourly_data],axis =  1)
    hourly_data['zip'] = m
    Zip_Daily_Hourly_Solar_Production = pd.concat([Zip_Daily_Hourly_Solar_Production,hourly_data],axis =0)
Zip_Daily_Hourly_Solar_Production.rename(columns = {0:'Production'},inplace = True)

27525


NameError: name 'PySSC' is not defined

In [ ]:
Zip_Daily_Hourly_Solar_Production

In [ ]:
Demand Data =

BA, Month, Hour (average)

PV:

Zip, Month, Day, Hour 


In [ ]:
Zip_Hourly_Solar_Production = Zip_Daily_Hourly_Solar_Production.groupby(['zip','month','hour'])['Production'].mean().reset_index()

# Final Merge

In [ ]:
Scheduel_Demand_Production_DF = Demand_Rate_Zip_Schedule.merge(Zip_Hourly_Solar_Production,on = ['zip','month','hour'],how = 'left')

In [ ]:
Scheduel_Demand_Production_DF

In [ ]:
Scheduel_Demand_Production_DF.to_csv('Scheduel_demand_Production.csv')